In [ ]:
### Install Qiskit, if needed

%pip install qiskit[visualization]==1.0.2
%pip install qiskit_aer
%pip install qiskit_ibm_runtime
%pip install matplotlib
%pip install pylatexenc
%pip install qiskit-transpiler-service
%pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git

In [1]:
# imports ()
import random
import numpy as np
from typing import List, Callable
from scipy.optimize import minimize
from scipy.optimize._optimize import OptimizeResult
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from qiskit.primitives import StatevectorSampler, PrimitiveJob
from qiskit.circuit.library import TwoLocal, MCXGate, MCMT, ZGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_aer import AerSimulator

import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

In [3]:
def check_accuracy(num_trials=10):
    results = []
    for _ in range(num_trials):
        main(results)
        
    count = 0
    for res in results:
        if res:
            count += 1
    
        
    print('Accuracy: {}'.format(count / len(results)))

In [4]:
def set_io_qubits(qubit_count):
    """Add the specified number of input and output qubits."""
    return QuantumCircuit(qubit_count + 1, qubit_count)

def make_oracle(circuit, qubit_count, x_bits):
    """Implement function {f(x) = 1 if x==x', f(x) = 0 if x!= x'}."""
    # Apply X gates to match x_bits
    for i in range(qubit_count):
        if x_bits[i] == 0:
            circuit.x(i)
            
    circuit.compose(MCMT(ZGate(), qubit_count - 1, 1), inplace=True)
    
    # Apply X gates to reset the original state
    for i in range(qubit_count):
        if x_bits[i] == 0:
            circuit.x(i)

def make_grover_circuit(qubit_count, x_bits):
    """Find the v alue recognized by the oracle in sqrt(N) attempts."""
    # Initialize circuit
    circuit = set_io_qubits(qubit_count)
    
    # Initialize qubits
    circuit.h(range(qubit_count))
    circuit.x(qubit_count)
    
    # Add oracle
    make_oracle(circuit, qubit_count, x_bits)
    
    # Grover operator (Diffusion operator)
    circuit.h(range(qubit_count))
    circuit.x(range(qubit_count))
    
    circuit.compose(MCMT(ZGate(), qubit_count - 1, 1), inplace=True)
    
    circuit.x(range(qubit_count))
    circuit.h(range(qubit_count))
    
    # Measure
    circuit.measure(range(qubit_count), range(qubit_count))
    
    return circuit

In [28]:
def main(solution=False, qubit_count=3, shots=1024, results=[], output=False):
    if not solution:
        # Choose the x' and make an oracle which can recognize it.
        x_bits = [random.randint(0, 1) for _ in range(qubit_count)]
    else:
        x_bits = solution
        
    rev_x_bits = x_bits[::-1]

    # Make Grover's circuit
    circuit = make_grover_circuit(qubit_count, rev_x_bits)

    # Simulate the circuit
    simulator = FakeSherbrooke()
    compiled_circuit = transpile(circuit, simulator, optimization_level=3)
    result = simulator.run(compiled_circuit, shots=shots).result()
    counts = result.get_counts()

    # Check if we actually found the secret value
    most_common_bitstring = max(counts, key=counts.get)

    # prints information
    if (output):
        print('Secret bit sequence: {}'.format(x_bits))
        print('Circuit:')
        print(circuit)
        print('Sampled results:\n{}'.format(counts))
        print('Most common bitstring: {}'.format(most_common_bitstring))
        print('Found a match: {}'.format(
        most_common_bitstring == ''.join(map(str, x_bits))))
    else:
        results.append(most_common_bitstring == ''.join(map(str, x_bits)))

In [31]:
if __name__ == '__main__':
    main(solution=[0, 1, 0], output=True)

Secret bit sequence: [0, 1, 0]
Circuit:
     ┌───┐┌───┐   ┌───┐┌───┐┌───┐   ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├┤M├──────
     ├───┤└───┘ │ ├───┤├───┤└───┘ │ ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├──────■─┤ H ├┤ X ├──────■─┤ X ├┤ H ├─╫─┤M├───
     ├───┤┌───┐ │ ├───┤├───┤┌───┐ │ ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├─╫──╫─┤M├
     ├───┤└───┘   └───┘└───┘└───┘   └───┘└───┘ ║  ║ └╥┘
q_3: ┤ X ├─────────────────────────────────────╫──╫──╫─
     └───┘                                     ║  ║  ║ 
c: 3/══════════════════════════════════════════╩══╩══╩═
                                               0  1  2 
Sampled results:
{'010': 707, '001': 44, '011': 49, '110': 39, '111': 56, '100': 47, '000': 43, '101': 39}
Most common bitstring: 010
Found a match: True


In [22]:
for i in range(10):
    main(output=True)
    print('\n')

Secret bit sequence: [1, 0, 0]
Circuit:
     ┌───┐┌───┐   ┌───┐┌───┐┌───┐   ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├┤M├──────
     ├───┤├───┤ │ ├───┤├───┤├───┤ │ ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├─╫─┤M├───
     ├───┤└───┘ │ ├───┤├───┤└───┘ │ ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├──────■─┤ H ├┤ X ├──────■─┤ X ├┤ H ├─╫──╫─┤M├
     ├───┤        └───┘└───┘        └───┘└───┘ ║  ║ └╥┘
q_3: ┤ X ├─────────────────────────────────────╫──╫──╫─
     └───┘                                     ║  ║  ║ 
c: 3/══════════════════════════════════════════╩══╩══╩═
                                               0  1  2 
Sampled results:
{'100': 694, '000': 49, '011': 46, '110': 62, '010': 34, '111': 43, '101': 46, '001': 50}
Most common bitstring: 100
Found a match: True


Secret bit sequence: [1, 1, 0]
Circuit:
     ┌───┐┌───┐   ┌───┐┌───┐┌───┐   ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├┤M├──────
     ├───┤└───┘ │ ├───┤├───┤└───┘ │ 

In [ ]:
check_accuracy(100)